# Практика 6: Атака по переносу (Transfer Attack) на модели ИИ

Картунчиков Артем ББМО-01-23

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np

# Загрузка данных MNIST

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть

In [3]:

model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])
# Компиляция модели

c:\Users\artem\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
# Обучение модели
model1.fit(train_images, train_labels, epochs=5)
# Сохранение модели
model1.save('mnist_model1.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 982us/step - accuracy: 0.8773 - loss: 0.4358
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9661 - loss: 0.1177
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 968us/step - accuracy: 0.9767 - loss: 0.0741
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 969us/step - accuracy: 0.9832 - loss: 0.0544
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 999us/step - accuracy: 0.9876 - loss: 0.0398


# Модель 2: Свёрточная нейронная сеть (CNN)

In [5]:
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])
# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
# Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
# Сохранение модели
model2.save('mnist_model2.h5')

c:\Users\artem\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9088 - loss: 0.2919
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9843 - loss: 0.0521
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9911 - loss: 0.0313
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9938 - loss: 0.0209
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9959 - loss: 0.0134


# Реализация атаки FGSM на первую модель

## Функции для реализации FGSM атаки

In [6]:
def fgsm_attack(image, epsilon, gradient):
  # Применение знака градиента к изображению
  perturbed_image = image + epsilon * np.sign(gradient)
  # Обрезка значений, чтобы они оставались в пределах [0,1]
  perturbed_image = np.clip(perturbed_image, 0, 1)
  return perturbed_image

In [7]:
def generate_adversarial_example(model, image, label, epsilon):
    # Превращаем изображение в формат, подходящий для модели
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))

    # Если label — это one-hot вектор, преобразуем его в индекс
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)

    gradient = tape.gradient(loss, image)

    # Применяем FGSM
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())

    # Убедимся, что adversarial_image имеет правильную форму
    return np.reshape(adversarial_image, (28, 28, 1))

In [8]:
def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))

    adversarial_images = np.array(adversarial_images)

    # Проверка формы
    print("Shape of adversarial_images:", adversarial_images.shape)

    return adversarial_images

## Генерация противоречивых примеров для первой модели

In [9]:
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)

Shape of adversarial_images: (10000, 28, 28)


# Оценка противоречивых примеров на обеих моделях

## Оценка первой модели на противоречивых примерах

In [10]:
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.0985 - loss: 5.9598
Accuracy of model1 on adversarial examples: 0.12929999828338623


## Оценка второй модели на противоречивых примерах (перенос атаки)

In [11]:
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9526 - loss: 0.1562
Accuracy of model2 on adversarial examples from model1: 0.9617000222206116


# Анализ переносимости атак

In [12]:
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28,
28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.9297 - loss: 0.2365
Accuracy of model1 on adversarial examples from model2: 0.9420999884605408


# Выводы

В ходе работы было исследовано влияние атаки переноса, при которой примеры, созданные для одной модели, использовались для атаки на другую. Для этой цели был применён метод FGSM (Fast Gradient Sign Method), позволяющий быстро генерировать искажающие примеры.

Результаты эксперимента показали, что модель 1, на которой были созданы противоречивые примеры с использованием FGSM, продемонстрировала значительное снижение точности с 98% до 13%, что свидетельствует о её высокой уязвимости. Модель 2 оказалась гораздо более устойчивой, её точность снизилась всего на 4%, что указывает на её лучшую защиту от переноса атак.

Атака переноса с применением FGSM может значительно ухудшить точность модели, особенно если примеры сгенерированы специально для неё. В то же время другая модель может оказаться менее подверженной таким атакам. Эти результаты подчёркивают важность разработки более устойчивых моделей, способных сохранять точность при атаке, перенесённой с другой модели.